# 04 - Regime Detection

This notebook implements Hidden Markov Model for market regime classification.

In [ ]:
import sys
sys.path.append('../src')

from regime import RegimeDetector, detect_regimes
import pandas as pd
import numpy as np

In [ ]:
# Load Features
data_dir = '../data'
df = pd.read_csv(f'{data_dir}/nifty_features_5min.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
print(f"Loaded {len(df)} rows")

In [ ]:
# HMM Features (Options-based)
hmm_features = [
    'avg_iv', 'iv_spread', 'pcr_oi',
    'call_delta', 'call_gamma', 'call_vega',
    'futures_basis', 'spot_returns'
]

# Check available features
available = [f for f in hmm_features if f in df.columns]
print(f"Available HMM features: {available}")

In [ ]:
# Detect Regimes
df = detect_regimes(df, available)

# Regime Distribution
print("\n=== Regime Distribution ===")
print(df['regime'].value_counts())

In [ ]:
# Regime Statistics
print("\n=== Regime Statistics ===")
regime_stats = df.groupby('regime').agg({
    'close_spot': ['mean', 'std'],
    'spot_returns': 'mean'
})
print(regime_stats)

In [ ]:
# Save with regimes
df.to_csv(f'{data_dir}/nifty_regimes_5min.csv', index=False)
print("Saved data with regimes")

In [ ]:
# Visualize (if matplotlib available)
try:
    import matplotlib.pyplot as plt
    
    sample = df.iloc[::10]  # Sample for speed
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    colors = {1: 'green', -1: 'red', 0: 'yellow'}
    for regime in [1, -1, 0]:
        mask = sample['regime'] == regime
        ax.scatter(sample.loc[mask, 'timestamp'], sample.loc[mask, 'close_spot'],
                  c=colors[regime], s=1, label=f'Regime {regime}', alpha=0.5)
    
    ax.set_xlabel('Time')
    ax.set_ylabel('Price')
    ax.set_title('NIFTY Price with Regime Overlay')
    ax.legend()
    
    plt.savefig('../plots/regime_overlay.png', dpi=100, bbox_inches='tight')
    plt.show()
except ImportError:
    print("matplotlib not available for plotting")